# 3D реконструкцiя зображення 2D


### 1. Заватаження стороннiх бiблiотек

In [1]:
# Dependencies installation

!pip install torch torchvision torchaudio
!pip install Pillow
!pip install transformers
!pip install open3d
!pip install matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
# Importing libraries

import matplotlib.pyplot as plt
import open3d as o3d
from PIL import Image
import numpy as np
import torch

# We are directly downloading a model from huggingface which will do the heavy lifting and we dont need to train our own model
from transformers import DPTImageProcessor, DPTForDepthEstimation


### 2. Loading model

In [3]:
# завантаження моделi

image_processor = DPTImageProcessor.from_pretrained("Intel/dpt-hybrid-midas")
model = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas", low_cpu_mem_usage=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/9.88k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/490M [00:00<?, ?B/s]

### 3. Завнтаження зображення

In [4]:
# Завантаження зображення

name = "heart"
image = Image.open(name + ".jpg")


FileNotFoundError: [Errno 2] No such file or directory: 'heart.jpg'

In [ ]:
# Змiна розмiру зображення

new_height = 1024 if image.height > 1024 else image.height
new_height -= (new_height % 32)
new_width = int(new_height * image.width / image.height)
diff = new_width % 32
new_width = new_width - diff if diff < 16 else new_width + diff
new_size = (new_width, new_height)
image = image.resize(new_size)



### 4. Пiдготовка зображення до обробки моделю

In [ ]:
inputs = image_processor(images=image, return_tensors="pt")

### 5. Генерацiя передбачення моделю машинного навчання

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
    predicted_depth = outputs.predicted_depth

### 6. Пост-обробка передбачення моделi

In [ ]:

pad= 2
output = predicted_depth.squeeze().cpu().numpy() * 1000.0
output = output[pad:-pad, pad:-pad]
image = image.crop((pad, pad, image.width - pad, image.height - pad))


### 7. Генарацiя результатiв

In [ ]:

fig, ax = plt.subplots(1, 2)
ax[0].imshow(image)
ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
ax[1].imshow(output, cmap= 'Spectral')
ax[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.tight_layout()
plt.pause(5)




### 8.  Пiдготовка аналiзу глибини зображення для open3d

In [ ]:

width, height = image.size

depth_image = (output * 255 / np.max(output)).astype( 'uint8')

# Resizing depth_image to match color image dimensions
depth_image = Image.fromarray(depth_image).resize(image.size)
depth_image = np.array(depth_image)
image = np.array(image)

# create rgbd image
depth_o3d = o3d.geometry.Image(depth_image)
image_o3d = o3d.geometry.Image(image)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d, depth_o3d, convert_rgb_to_intensity=False)


### 9. Створення камери

In [ ]:
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera_intrinsic.set_intrinsics(width, height, 500, 500, width / 2, height / 2)


### 10. Створення та збереження хмарини точок

In [ ]:
pcd_raw = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)


In [ ]:
# outlier removal
cl, ind = pcd_raw.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd = pcd_raw.select_by_index(ind)

In [ ]:
# estimate normals

pcd.estimate_normals()
pcd.orient_normals_to_align_with_direction()

In [ ]:
import os

# Create output directory
output_dir = "pointclouds"
os.makedirs(output_dir, exist_ok=True)

# Define file path
pcd_file = os.path.join(output_dir, name + "pcd.ply")

# Save the point cloud so that we can see it in any pointcloud viewer
o3d.io.write_point_cloud(pcd_file, pcd)

print(f"Point cloud saved to: {pcd_file}")